In [1]:
import numpy as np
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn import metrics
np.random.seed(13)
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from stockstats import StockDataFrame 
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data

import dataPreparation
files=['NDX']#,'AMZN','GOOGL','MSFT','FB','INTC','CSCO','CMCSA','NVDA','NFLX','ADBE','AMGN','TXN','AVGO','PYPL','GILD','COST','QCOM']       
features=[  2,   5,   3 , 37 ,  6 ,  7 ,  0 ,  8  ,59 ,  9 , 10 , 11 , 12  ,13   ,1 , 44  ,42,  60,
  46 , 21 , 78,  45 , 80 , 82 , 84 , 86 , 88 , 90 , 92,  23,  71,  41, 110 , 40, 108,  39,
  62, 106 , 38, 109]
prices=list()
for file in files:
   
    priceData =  pd.read_csv('/home/andrea/Desktop/NLFF/YahooData/'+file+'.csv')
    priceData=priceData.drop(['Adj Close'],axis=1)
   
    
    string=['close_2_sma','close_4_sma','close_6_sma','close_8_sma','close_10_sma','close_12_sma','close_14_sma','close_16_sma','close_18_sma','pdi','mdi','dx','adx','adxr','cr','cr-ma1','cr-ma2','cr-ma3','boll','boll_ub','boll_lb','volume_-2_d','volume_-4_d','volume_-6_d','volume_-8_d','volume_-10_d','volume_-12_d','macd','macds','macdh','kdjk','kdjd','kdjj','tr','atr','wr_2','wr_10','wr_6','wr_8','wr_12','wr_14','close_-2_d','close_-4_d','close_-6_d','close_-8_d','close_-10_d','close_-12_d','close_-14_d','close_-16_d','rsi_2','rsi_4','rsi_6','rsi_8','rsi_10','rsi_12','rsi_14','rsi_16']
    stock = StockDataFrame.retype(priceData)
    
    stock.get(string)
    price = stock
    
    print(price.iloc[:,features])
    
    #['macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6''close_10_sma''tr''middle_{}_sma'
    #df['dma'] = df['close_10_sma'] - df['close_50_sma'] pdi'] = cls._get_pdi(df, 14)df['mdi'] = cls._get_mdi(df, 14)
    #df['dx'] = cls._get_dx(df, 14) df['adx'] = df['dx_6_ema'] df['adxr'] = df['adx_6_ema']]
    #

    prices.append(price.iloc[:,features])

/home/andrea/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


                    low  close_2_sma        close       middle  close_4_sma  \
date                                                                          
1985-10-01   110.565002   112.139999   112.139999   111.621668   112.139999   
1985-10-02   110.779999   111.482498   110.824997   111.381666   111.482498   
1985-10-03   110.120003   110.847500   110.870003   110.725001   111.278333   
1985-10-04   109.855003   110.472500   110.074997   110.266668   110.977499   
1985-10-07   108.175003   109.137497   108.199997   108.836667   109.992499   
1985-10-08   106.750000   107.680001   107.160004   107.393334   109.076250   
1985-10-09   107.114998   107.895001   108.629997   108.244998   108.516249   
1985-10-10   107.809998   109.032498   109.434998   108.918333   108.356249   
1985-10-11   109.425003   110.242501   111.050003   110.531670   109.068750   
1985-10-14   111.050003   112.117501   113.184998   112.501666   110.574999   
1985-10-15   112.790001   113.282498   113.379997   

In [2]:
def smote(x,y):
    X_resampled, y_resampled = SMOTE().fit_sample(x, y)
    #print('check',sum(y_resampled)/len(y_resampled))
    return X_resampled,y_resampled
def balance(x,y):
    posindex=np.where( y == 1 )
    negindex=np.where( y == 0 )
    xt=[]
    yt=[]
    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))

    #for i in range(1,nindex):
    yt=np.concatenate((y[posindex[0][0:nindex]],y[negindex[0][0:nindex]]))
    xt=np.concatenate((x[posindex[0][0:nindex]],x[negindex[0][0:nindex]]))
    
    return xt,yt

def balanceup(x,y):
    posindex=np.where( y == 1 )
    negindex=np.where( y == 0 )
    xt=[]
    yt=[]
    yindex=[]
    
    if(len(posindex[0])!=0 and len(negindex[0])!=0):
       
        nindex=max(len(posindex[0]),len(negindex[0]))
        mini=min(len(posindex[0]),len(negindex[0]))
        diff=nindex-mini
        u=0
        for i in range(0,mini):
            yt.append(y[posindex[0][i]])
            yt.append(y[negindex[0][i]])
            xt.append(x[posindex[0][i]])
            xt.append(x[negindex[0][i]])
        #print('first',sum(yt)/len(yt)) 
        if(len(posindex[0])>len(negindex[0])):
            toextract=negindex
            enter=posindex
        else:
            toextract=posindex
            enter=negindex
        if(diff!=0 and len(toextract[0])!=0):
            for i in range(0,diff):
                r=np.random.randint(0,len(toextract))
                yt.append(y[toextract[0][r]])
                xt.append(x[toextract[0][r]])
                yt.append(y[enter[0][mini+i]])
                xt.append(x[enter[0][mini+i]])
    else:
        #print('Unbalance')
        u=1
        xt=x
        yt=y
    #print(sum(yt)/len(yt))              
    return xt,yt,u
                    

In [4]:
o=0
accstocksresults=[]
f1stocksresults=[]
f1scoreresults=[]
aucresults=[]

for price in prices:
    #print('New price[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]')
    
   
    print('Working on...',files[o])
    o+=1
    xtemp=price
    xtemp=np.nan_to_num(np.asarray(xtemp, dtype=float))
    xtemp=normalize(xtemp,axis=0,norm='max')
    print(xtemp)
    accintime=[]
    trendwindowtime=[1,2,4,5,10,15,30]
    f1total=[]
    acctotal=[]
    auctotal=[]
    f1scoretotal=[]
    for t in trendwindowtime:
        #1
    #label because of the maket and append values without data
    #simo theroy past trend
        
        x=[]
        y=[]
        

        #print('============================================================')
        #print('Working on window:',t)
        #print(len(xtemp))
        ##QUI C E L'UNICO APPUNTO GUARDA SE CON +1 CAMBIA
        for i in range(0,len(price)-t-1):
            s=np.sign(price.iloc[i+t+1]['close']-price.iloc[i+1]['open'])
            
            if(s==-1):
                y.append(0)
            else:
                y.append(1)
            x.append(xtemp[i])
        y=np.array(y)
       
        x=np.array(x)
        permindex=range(0,len(x))
        #permindex=np.random.permutation(permindex)
        train=0.8
        nt=math.ceil(len(x)*train)
        trainvalindex=permindex[0:nt]
        testindex=permindex[nt:]
        x_tv=[]
        y_tv=[]
        x_test=[]
        y_test=[]
        x_tv=x[trainvalindex]
        y_tv=y[trainvalindex]
        x_test=x[testindex]
        y_test=y[testindex]
    

        cspace=np.concatenate(((np.logspace(0,3,15)),np.logspace(-4,4,10)))
        gspace=[1]
        bestsvm=None
        maxacc=0
        cvacc=0
        maxg=0
        maxc=0
        #print('Model Selection...')
        #model selection
        cvacc=0
        totu=0
        for c in cspace:
            #print()
            for g in gspace:
                
                cvacclist=[]
                #faccio cross validation
                #start with 40% as train and 10% for validation and then i move in percentege
                # 0-40 40-50
                # 0-50 50-60
                # 0-60 60-70 
                #etc test set is completely external i do in some way error extimation changin the ticker
                trainpoint=math.floor(len(x_tv)*0.40)
                dimval=math.floor(trainpoint*0.25)
                endval=trainpoint+dimval
                
                for i in range(0,6):
                    #print('-----')
                    x_train=x_tv[0:trainpoint]
                    y_train=y_tv[0:trainpoint]
                    x_val=x_tv[trainpoint:endval]
                    y_val=y_tv[trainpoint:endval]
                    #print(trainpoint)
                    #print(endval)
                    #print(len(x_tv))
                    trainpoint=trainpoint+dimval
                    endval=endval+dimval
                    p=sum(y_train)/(len(y_train)-sum(y_train))
                    rbf_svm=svm.SVC(C=c)
                    x_train,y_train,e=balanceup(x_train,y_train)
                    rbf_svm.fit(x_train,y_train)
                    #prediction=rbf_svm.predict(x_val)
                    x_val,y_val,u=balanceup(x_val,y_val)
                    totu=totu+u
                    cvacclist.append(rbf_svm.score(x_val,y_val))
                cvacc=sum(cvacclist)/len(cvacclist)
                if(cvacc>maxacc):
                    #print(cvacc)
                    maxacc=cvacc
                    
                    maxc=c
                    bestsvm=rbf_svm

        #print('ACCURACY on val:',maxacc)
        p=sum(y_tv)/(len(y_tv)-sum(y_tv))
        print(maxc)
        rbf_svm=svm.SVC(C=maxc)
        x_tv,y_tv=smote(x_tv,y_tv)
        rbf_svm.fit(x_tv,y_tv)
        prediction=rbf_svm.predict(x_test)
        confmatrix=confusion_matrix(y_test, prediction)
        print('Window:',t)
        print(confmatrix)
        CM = f1_score(y_test, prediction)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, prediction)
        auc=metrics.auc(fpr, tpr)
        valueacc=rbf_svm.score(x_test,y_test)
        #print('Accuracy on TEST set:',testacc
        
        print('F1,score',CM)
        print('Accuracy:',valueacc)
        f1scoretotal.append(CM)
        f1total.append(totu)
        acctotal.append(valueacc)
        auctotal.append(auc)
    print(f1total)
    print(acctotal)
    print('accuracy')
    print('accuracy')
    f1total=normalize(np.asarray(f1total).reshape(-1,1),axis=0,norm='max')
    plt.plot(trendwindowtime,acctotal,'g')
    plt.plot(trendwindowtime,f1total,'r')
    plt.show()
    f1scoreresults.append(f1scoretotal)
    accstocksresults.append(acctotal)
    f1stocksresults.append(f1total)
    aucresults.append(auctotal)
for p in accstocksresults:
    plt.plot(p)
plt.show()

Working on... NDX
[[0.01528471 0.01542997 0.01540471 ... 0.         0.         0.        ]
 [0.01531443 0.0153395  0.01522407 ... 0.         0.25891448 0.        ]
 [0.01522319 0.01525213 0.01523025 ... 0.03975218 0.10360722 0.00544996]
 ...
 [0.9943763  0.99168168 0.98978925 ... 0.78164059 0.43453404 0.31010226]
 [1.         0.99652571 1.         ... 0.83134288 0.49456261 0.36536905]
 [0.99824018 1.         0.99672645 ... 0.78886382 0.42721092 0.32431127]]
1.0
Window: 1
[[689   0]
 [959   0]]


/home/andrea/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1,score 0.0
Accuracy: 0.41808252427184467
1.0
Window: 2
[[634  47]
 [915  52]]
F1,score 0.0975609756097561
Accuracy: 0.4162621359223301
1291.5496650148827
Window: 4
[[404 237]
 [624 383]]
F1,score 0.47080516287645974
Accuracy: 0.47754854368932037
1000.0
Window: 5
[[592  33]
 [961  62]]
F1,score 0.11091234347048302
Accuracy: 0.39684466019417475
0.0001
Window: 10
[[ 606    0]
 [1041    0]]


/home/andrea/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1,score 0.0
Accuracy: 0.3679417122040073


KeyboardInterrupt: 

In [ ]:
tempacc=aucresults


In [ ]:
import operator
t=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(0,len(tempacc)):
    #for k in range(0,len(tempacc[i])):
        #print(tempacc[i][k])
     #   if(tempacc[i][k]<0.5):
      #     tempacc[i][k]=1-tempacc[i][k]
    t=np.add(t,tempacc[i])
    plt.plot(tempacc[i])
    
print(t/len(tempacc))
plt.show()

In [ ]:
plt.plot(trendwindowtime,t/len(tempacc))
plt.show()

In [ ]:
Working on... AAPL
1.0
Window: 0
[[300 290]
 [286 279]]
F1,score 0.49206349206349204
Accuracy: 0.5012987012987012
10000.0
Window: 4
[[254 302]
 [274 324]]
F1,score 0.5294117647058822
Accuracy: 0.5008665511265165
10000.0
Window: 5
[[293 274]
 [286 301]]
F1,score 0.5180722891566264
Accuracy: 0.5147313691507799
7.196856730011519
Window: 25
[[396 139]
 [421 194]]
F1,score 0.40928270042194087
Accuracy: 0.5130434782608696
51.7947467923121
Window: 40
[[322 185]
 [400 240]]
F1,score 0.4507042253521127
Accuracy: 0.4899738448125545
227.58459260747887
Window: 125
[[380 120]
 [136 494]]
F1,score 0.7942122186495177
Accuracy: 0.7734513274336283
372.7593720314938
Window: 170
[[321 161]
 [133 506]]
F1,score 0.7748851454823888
Accuracy: 0.7377341659232828
10000.0
Window: 250
[[305 157]
 [103 540]]
F1,score 0.8059701492537313
Accuracy: 0.7647058823529411
51.7947467923121
Window: 290
[[425 108]
 [ 98 466]]
F1,score 0.8189806678383128
Accuracy: 0.8122151321786691
138.94954943731375
Window: 350
[[368 136]
 [174 407]]
F1,score 0.7241992882562277
Accuracy: 0.7142857142857143
166.81005372000558
Window: 400
[[369 155]
 [225 326]]
F1,score 0.6317829457364341
Accuracy: 0.6465116279069767
372.7593720314938
Window: 450
[[260 308]
 [246 251]]
F1,score 0.47537878787878785
Accuracy: 0.47981220657276996
[0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 25, 25]
[0.5012987012987012, 0.5008665511265165, 0.5147313691507799, 0.5130434782608696, 0.4899738448125545, 0.7734513274336283, 0.7377341659232828, 0.7647058823529411, 0.8122151321786691, 0.7142857142857143, 0.6465116279069767, 0.47981220657276996]
accuracy
accuracy

Working on... AMZN
1.637893706954064
Window: 0
[[336 249]
 [291 278]]
F1,score 0.5072992700729927
Accuracy: 0.5320623916811091
19.306977288832496
Window: 4
[[265 331]
 [226 332]]
F1,score 0.5438165438165438
Accuracy: 0.5173310225303293
19.306977288832496
Window: 5
[[258 337]
 [184 374]]
F1,score 0.5894405043341214
Accuracy: 0.5481352992194276
19.306977288832496
Window: 25
[[214 361]
 [157 417]]
F1,score 0.6168639053254438
Accuracy: 0.5491731940818103
31.622776601683793
Window: 40
[[ 53 530]
 [ 26 537]]
F1,score 0.6588957055214725
Accuracy: 0.5148342059336823
1000.0
Window: 125
[[289 220]
 [268 352]]
F1,score 0.5906040268456375
Accuracy: 0.5677590788308238
372.7593720314938
Window: 170
[[108 369]
 [ 86 557]]
F1,score 0.710006373486297
Accuracy: 0.59375
0.3593813663804626
Window: 250
[[  0 435]
 [  0 669]]
F1,score 0.754653130287648
Accuracy: 0.6059782608695652
227.58459260747887
Window: 290
[[ 33 367]
 [ 39 657]]
F1,score 0.7639534883720931
Accuracy: 0.6295620437956204
1.0
Window: 350
[[  0 413]
 [  0 671]]
F1,score 0.7646723646723648
Accuracy: 0.6190036900369004
2.782559402207126
Window: 400
[[  0 435]
 [  0 639]]
F1,score 0.7460595446584939
Accuracy: 0.5949720670391061
1.637893706954064
Window: 450
[[  0 416]
 [  0 648]]
F1,score 0.7570093457943925
Accuracy: 0.6090225563909775
[0, 0, 0, 0, 0, 0, 25, 50, 50, 75, 100, 75]
[0.5320623916811091, 0.5173310225303293, 0.5481352992194276, 0.5491731940818103, 0.5148342059336823, 0.5677590788308238, 0.59375, 0.6059782608695652, 0.6295620437956204, 0.6190036900369004, 0.5949720670391061, 0.6090225563909775]
accuracy
accuracy

Working on... GOOGL
227.58459260747887
Window: 0
[[291 280]
 [304 279]]
F1,score 0.4886164623467601
Accuracy: 0.49393414211438474
0.3593813663804626
Window: 4
[[399 185]
 [370 199]]
F1,score 0.4176285414480588
Accuracy: 0.5186470078057241
21.54434690031882
Window: 5
[[244 338]
 [228 343]]
F1,score 0.547923322683706
Accuracy: 0.5091066782307025
1.637893706954064
Window: 25
[[359 193]
 [465 132]]
F1,score 0.28633405639913234
Accuracy: 0.42732811140121846
372.7593720314938
Window: 40
[[182 377]
 [130 457]]
F1,score 0.6432090077410275
Accuracy: 0.5575916230366492
10000.0
Window: 125
[[184 376]
 [ 34 535]]
F1,score 0.722972972972973
Accuracy: 0.6368467670504872
10000.0
Window: 170
[[168 366]
 [ 28 558]]
F1,score 0.7390728476821192
Accuracy: 0.6482142857142857
2.782559402207126
Window: 250
[[180 349]
 [150 425]]
F1,score 0.6300963676797627
Accuracy: 0.5480072463768116
0.0001
Window: 290
[[300 283]
 [230 283]]
F1,score 0.5245597775718257
Accuracy: 0.531934306569343
0.000774263682681127
Window: 350
[[313 289]
 [153 329]]
F1,score 0.5981818181818181
Accuracy: 0.5922509225092251
0.000774263682681127
Window: 400
[[289 239]
 [121 425]]
F1,score 0.7024793388429752
Accuracy: 0.664804469273743
0.005994842503189409
Window: 450
[[313 183]
 [219 349]]
F1,score 0.6345454545454545
Accuracy: 0.6221804511278195
[0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 25, 25]
[0.49393414211438474, 0.5186470078057241, 0.5091066782307025, 0.42732811140121846, 0.5575916230366492, 0.6368467670504872, 0.6482142857142857, 0.5480072463768116, 0.531934306569343, 0.5922509225092251, 0.664804469273743, 0.6221804511278195]
accuracy
accuracy

Working on... MSFT
0.000774263682681127
Window: 0
[[427 144]
 [406 178]]
F1,score 0.39293598233995586
Accuracy: 0.5238095238095238
10000.0
Window: 4
[[326 220]
 [333 275]]
F1,score 0.4986400725294651
Accuracy: 0.5207972270363952
7.196856730011519
Window: 5
[[304 247]
 [295 308]]
F1,score 0.531951640759931
Accuracy: 0.5303292894280762
227.58459260747887
Window: 25
[[236 286]
 [263 365]]
F1,score 0.5707584050039093
Accuracy: 0.5226086956521739
31.622776601683793
Window: 40
[[274 240]
 [286 347]]
F1,score 0.5688524590163934
Accuracy: 0.5414123801220575
0.000774263682681127
Window: 125
[[349 116]
 [472 193]]
F1,score 0.3963039014373717
Accuracy: 0.479646017699115
0.000774263682681127
Window: 170
[[353 113]
 [476 179]]
F1,score 0.3780359028511087
Accuracy: 0.4745762711864407
21.54434690031882
Window: 250
[[200 209]
 [240 456]]
F1,score 0.6700955180014695
Accuracy: 0.5936651583710407
0.005994842503189409
Window: 290
[[270 157]
 [326 344]]
F1,score 0.5875320239111871
Accuracy: 0.5597082953509571
0.000774263682681127
Window: 350
[[285 116]
 [403 281]]
F1,score 0.5198889916743756
Accuracy: 0.5216589861751152
2.782559402207126
Window: 400
[[325  38]
 [542 170]]
F1,score 0.3695652173913044
Accuracy: 0.4604651162790698
0.000774263682681127
Window: 450
[[189 112]
 [443 321]]
F1,score 0.5363408521303258
Accuracy: 0.4788732394366197
[0, 0, 0, 0, 0, 0, 25, 25, 25, 25, 25, 25]
[0.5238095238095238, 0.5207972270363952, 0.5303292894280762, 0.5226086956521739, 0.5414123801220575, 0.479646017699115, 0.4745762711864407, 0.5936651583710407, 0.5597082953509571, 0.5216589861751152, 0.4604651162790698, 0.4788732394366197]
accuracy
accuracy

Working on... FB
/home/andrea/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
0.000774263682681127
Window: 0
[[340 221]
 [350 244]]
F1,score 0.46081208687440983
Accuracy: 0.5056277056277056
7.196856730011519
Window: 4
[[223 354]
 [176 401]]
F1,score 0.6021021021021021
Accuracy: 0.5407279029462738
0.046415888336127774
Window: 5
[[367 226]
 [291 270]]
F1,score 0.5108798486281929
Accuracy: 0.5519930675909879
372.7593720314938
Window: 25
[[158 404]
 [185 403]]
F1,score 0.5777777777777778
Accuracy: 0.48782608695652174
166.81005372000558
Window: 40
[[160 377]
 [156 454]]
F1,score 0.6301179736294239
Accuracy: 0.5353095030514385
1.637893706954064
Window: 125
[[149 257]
 [140 584]]
F1,score 0.7463258785942491
Accuracy: 0.6486725663716815
51.7947467923121
Window: 170
[[120 228]
 [ 89 684]]
F1,score 0.8118694362017803
Accuracy: 0.7172167707404103
1.0
Window: 250
[[136 195]
 [136 638]]
F1,score 0.7940261356565027
Accuracy: 0.7004524886877828
1.0
Window: 290
[[157 178]
 [132 630]]
F1,score 0.8025477707006371
Accuracy: 0.7174111212397447
1.0
Window: 350
[[213 117]
 [ 61 694]]
F1,score 0.8863346104725415
Accuracy: 0.8359447004608295
2.782559402207126
Window: 400
[[260  95]
 [ 32 688]]
F1,score 0.9155023286759814
Accuracy: 0.881860465116279
1.0
Window: 450
[[283  99]
 [ 20 663]]
F1,score 0.9176470588235294
Accuracy: 0.888262910798122
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50]
[0.5056277056277056, 0.5407279029462738, 0.5519930675909879, 0.48782608695652174, 0.5353095030514385, 0.6486725663716815, 0.7172167707404103, 0.7004524886877828, 0.7174111212397447, 0.8359447004608295, 0.881860465116279, 0.888262910798122]
accuracy
accuracy

Working on... INTC
610.5402296585327
Window: 0
[[426 131]
 [452 146]]
F1,score 0.33371428571428574
Accuracy: 0.49523809523809526
7.196856730011519
Window: 4
[[520  24]
 [602   8]]
F1,score 0.024922118380062305
Accuracy: 0.45753899480069327
227.58459260747887
Window: 5
[[492  54]
 [540  68]]
F1,score 0.1863013698630137
Accuracy: 0.4852686308492201
4.39397056076079
Window: 25
[[463   6]
 [656  25]]
F1,score 0.0702247191011236
Accuracy: 0.42434782608695654
1000.0
Window: 40
[[441  36]
 [589  81]]
F1,score 0.20584498094027953
Accuracy: 0.45510026155187444
1.0
Window: 125
[[443   0]
 [687   0]]
/home/andrea/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
F1,score 0.0
Accuracy: 0.3920353982300885
2.6826957952797255
Window: 170
[[444   4]
 [647  26]]
F1,score 0.07396870554765292
Accuracy: 0.4192685102586976
10000.0
Window: 250
[[363  35]
 [343 364]]
F1,score 0.6582278481012658
Accuracy: 0.6579185520361991
84.83428982440716
Window: 290
[[374   2]
 [515 206]]
F1,score 0.4434876210979548
Accuracy: 0.528714676390155
138.94954943731375
Window: 350
[[384   0]
 [542 159]]
F1,score 0.3697674418604651
Accuracy: 0.5004608294930876
166.81005372000558
Window: 400
[[312   0]
 [581 182]]
F1,score 0.38518518518518524
Accuracy: 0.4595348837209302
166.81005372000558
Window: 450
[[261   0]
 [572 232]]
F1,score 0.4478764478764479
Accuracy: 0.4629107981220657
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 25]
[0.49523809523809526, 0.45753899480069327, 0.4852686308492201, 0.42434782608695654, 0.45510026155187444, 0.3920353982300885, 0.4192685102586976, 0.6579185520361991, 0.528714676390155, 0.5004608294930876, 0.4595348837209302, 0.4629107981220657]
accuracy
accuracy

Working on... CSCO
10000.0
Window: 0
[[350 221]
 [352 231]]
F1,score 0.4463768115942029
Accuracy: 0.5034662045060658
21.54434690031882
Window: 4
[[404 138]
 [431 181]]
F1,score 0.388829215896885
Accuracy: 0.5069324090121318
10000.0
Window: 5
[[338 222]
 [365 228]]
F1,score 0.4372003835091083
Accuracy: 0.4908933217692975
2.782559402207126
Window: 25
[[441 122]
 [399 187]]
F1,score 0.41787709497206704
Accuracy: 0.546562228024369
19.306977288832496
Window: 40
[[357 211]
 [301 277]]
F1,score 0.5196998123827393
Accuracy: 0.5532286212914486
1.0
Window: 125
[[280 249]
 [311 289]]
F1,score 0.5079086115992969
Accuracy: 0.503985828166519
0.3593813663804626
Window: 170
[[161 379]
 [150 430]]
F1,score 0.619150467962563
Accuracy: 0.5276785714285714
1.637893706954064
Window: 250
[[103 386]
 [ 46 569]]
F1,score 0.7248407643312103
Accuracy: 0.6086956521739131
10000.0
Window: 290
[[332 157]
 [203 404]]
F1,score 0.6917808219178082
Accuracy: 0.6715328467153284
227.58459260747887
Window: 350
[[376  85]
 [278 345]]
F1,score 0.6552706552706552
Accuracy: 0.665129151291513
372.7593720314938
Window: 400
[[415  51]
 [242 366]]
F1,score 0.7141463414634146
Accuracy: 0.7271880819366853
4.39397056076079
Window: 450
[[100 320]
 [ 94 550]]
F1,score 0.726552179656539
Accuracy: 0.6109022556390977
[0, 0, 0, 0, 0, 0, 0, 50, 50, 50, 50, 50]
[0.5034662045060658, 0.5069324090121318, 0.4908933217692975, 0.546562228024369, 0.5532286212914486, 0.503985828166519, 0.5276785714285714, 0.6086956521739131, 0.6715328467153284, 0.665129151291513, 0.7271880819366853, 0.6109022556390977]
accuracy
accuracy

Working on... CMCSA
11.787686347935873
Window: 0
[[344 230]
 [307 273]]
F1,score 0.5041551246537396
Accuracy: 0.5346620450606586
2.6826957952797255
Window: 4
[[460 117]
 [408 169]]
F1,score 0.39165701042873696
Accuracy: 0.5450606585788561
4.39397056076079
Window: 5
[[436 151]
 [370 196]]
F1,score 0.42935377875136915
Accuracy: 0.5481352992194276
31.622776601683793
Window: 25
[[129 463]
 [118 439]]
F1,score 0.6017820424948594
Accuracy: 0.49434290687554394
138.94954943731375
Window: 40
[[181 405]
 [167 393]]
F1,score 0.5787923416789397
Accuracy: 0.5008726003490401
0.3593813663804626
Window: 125
[[ 54 545]
 [ 38 492]]
F1,score 0.6279514996809189
Accuracy: 0.48361381753764393
1.637893706954064
Window: 170
[[ 57 644]
 [ 43 376]]
F1,score 0.5225851285615011
Accuracy: 0.38660714285714287
0.3593813663804626
Window: 250
[[ 73 645]
 [ 20 366]]
F1,score 0.5239799570508232
Accuracy: 0.3976449275362319
0.3593813663804626
Window: 290
[[ 68 733]
 [ 13 282]]
F1,score 0.43053435114503813
Accuracy: 0.3193430656934307
0.0001
Window: 350
[[363 513]
 [ 28 180]]
F1,score 0.39955604883462814
Accuracy: 0.5009225092250923
0.000774263682681127
Window: 400
[[ 51 857]
 [  8 158]]
F1,score 0.26756985605419137
Accuracy: 0.1945996275605214
1.0
Window: 450
[[  0 952]
 [  0 112]]
F1,score 0.1904761904761905
Accuracy: 0.10526315789473684
[0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 75, 75]
[0.5346620450606586, 0.5450606585788561, 0.5481352992194276, 0.49434290687554394, 0.5008726003490401, 0.48361381753764393, 0.38660714285714287, 0.3976449275362319, 0.3193430656934307, 0.5009225092250923, 0.1945996275605214, 0.10526315789473684]
accuracy
accuracy

Working on... NVDA
/home/andrea/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
7.196856730011519
Window: 0
[[410 168]
 [389 188]]
F1,score 0.4030010718113612
Accuracy: 0.5177489177489177
0.005994842503189409
Window: 4
[[417 148]
 [391 198]]
F1,score 0.4235294117647059
Accuracy: 0.5329289428076257
2.6826957952797255
Window: 5
[[385 188]
 [340 241]]
F1,score 0.4772277227722772
Accuracy: 0.5424610051993067
1.637893706954064
Window: 25
[[395 146]
 [353 256]]
F1,score 0.506429277942631
Accuracy: 0.5660869565217391
2.6826957952797255
Window: 40
[[492  45]
 [454 156]]
F1,score 0.3847102342786683
Accuracy: 0.5649520488230165
0.000774263682681127
Window: 125
[[582  33]
 [449  66]]
F1,score 0.21498371335504884
Accuracy: 0.5734513274336284
0.005994842503189409
Window: 170
[[648   8]
 [460   5]]
F1,score 0.020920502092050212
Accuracy: 0.5825156110615521
10000.0
Window: 250
[[377 199]
 [110 419]]
F1,score 0.7306015693112468
Accuracy: 0.7203619909502262
10000.0
Window: 290
[[355 204]
 [117 421]]
F1,score 0.7239896818572656
Accuracy: 0.707383773928897
84.83428982440716
Window: 350
[[396 127]
 [222 340]]
F1,score 0.6608357628765792
Accuracy: 0.6783410138248848
166.81005372000558
Window: 400
[[337 201]
 [141 396]]
F1,score 0.6984126984126984
Accuracy: 0.6818604651162791
10000.0
Window: 450
[[253 260]
 [130 422]]
F1,score 0.6839546191247974
Accuracy: 0.6338028169014085
[0, 0, 0, 0, 0, 0, 0, 25, 25, 25, 25, 50]
[0.5177489177489177, 0.5329289428076257, 0.5424610051993067, 0.5660869565217391, 0.5649520488230165, 0.5734513274336284, 0.5825156110615521, 0.7203619909502262, 0.707383773928897, 0.6783410138248848, 0.6818604651162791, 0.6338028169014085]
accuracy
accuracy

Working on... NFLX
1291.5496650148827
Window: 0
[[251 333]
 [232 338]]
F1,score 0.5447219983883964
Accuracy: 0.5103986135181976
0.046415888336127774
Window: 4
[[ 16 592]
 [ 18 528]]
F1,score 0.6338535414165666
Accuracy: 0.4714038128249567
0.046415888336127774
Window: 5
[[  7 597]
 [  4 545]]
F1,score 0.644589000591366
Accuracy: 0.4787510841283608
1000.0
Window: 25
[[329 253]
 [333 234]]
F1,score 0.444022770398482
Accuracy: 0.4899912967798085
0.046415888336127774
Window: 40
[[522  25]
 [581  18]]
F1,score 0.056074766355140186
Accuracy: 0.4712041884816754
51.7947467923121
Window: 125
[[378 109]
 [379 263]]
F1,score 0.5187376725838263
Accuracy: 0.5677590788308238
10000.0
Window: 170
[[280 214]
 [192 434]]
F1,score 0.6813186813186812
Accuracy: 0.6375
10000.0
Window: 250
[[305 204]
 [182 413]]
F1,score 0.6815181518151815
Accuracy: 0.6503623188405797
10000.0
Window: 290
[[146 344]
 [ 46 560]]
F1,score 0.7417218543046358
Accuracy: 0.6441605839416058
0.046415888336127774
Window: 350
[[  1 471]
 [  2 610]]
F1,score 0.7206142941523922
Accuracy: 0.5636531365313653
51.7947467923121
Window: 400
[[  0 477]
 [  0 597]]
F1,score 0.7145421903052065
Accuracy: 0.5558659217877095
227.58459260747887
Window: 450
[[  0 463]
 [  0 601]]
F1,score 0.721921921921922
Accuracy: 0.5648496240601504
[0, 0, 0, 0, 0, 0, 25, 25, 25, 25, 75, 75]
[0.5103986135181976, 0.4714038128249567, 0.4787510841283608, 0.4899912967798085, 0.4712041884816754, 0.5677590788308238, 0.6375, 0.6503623188405797, 0.6441605839416058, 0.5636531365313653, 0.5558659217877095, 0.5648496240601504]
accuracy
accuracy

Working on... ADBE
/home/andrea/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
1.0
Window: 0
[[292 269]
 [319 274]]
F1,score 0.48239436619718307
Accuracy: 0.4904679376083189
51.7947467923121
Window: 4
[[398 132]
 [455 168]]
F1,score 0.3640303358613218
Accuracy: 0.4908933217692975
21.54434690031882
Window: 5
[[432 110]
 [457 154]]
F1,score 0.35200000000000004
Accuracy: 0.5082393755420642
2.6826957952797255
Window: 25
[[424  67]
 [543 115]]
F1,score 0.27380952380952384
Accuracy: 0.4691035683202785
1.637893706954064
Window: 40
[[ 67 414]
 [ 51 614]]
F1,score 0.7253396337861784
Accuracy: 0.5942408376963351
0.0001
Window: 125
[[  0 487]
 [  0 642]]
F1,score 0.7250141163184641
Accuracy: 0.5686448184233835
51.7947467923121
Window: 170
[[ 18 447]
 [ 31 624]]
F1,score 0.7230590961761297
Accuracy: 0.5732142857142857
31.622776601683793
Window: 250
[[ 84 353]
 [ 75 592]]
F1,score 0.7344913151364765
Accuracy: 0.6123188405797102
51.7947467923121
Window: 290
[[ 82 373]
 [125 516]]
F1,score 0.6745098039215687
Accuracy: 0.5456204379562044
7.196856730011519
Window: 350
[[194 300]
 [199 391]]
F1,score 0.6104605776736923
Accuracy: 0.5396678966789668
1291.5496650148827
Window: 400
[[ 10 348]
 [  2 714]]
F1,score 0.8031496062992126
Accuracy: 0.6741154562383612
610.5402296585327
Window: 450
[[  3 251]
 [  1 809]]
F1,score 0.8652406417112299
Accuracy: 0.7631578947368421
[0, 0, 0, 0, 0, 0, 0, 25, 25, 50, 50, 75]
[0.4904679376083189, 0.4908933217692975, 0.5082393755420642, 0.4691035683202785, 0.5942408376963351, 0.5686448184233835, 0.5732142857142857, 0.6123188405797102, 0.5456204379562044, 0.5396678966789668, 0.6741154562383612, 0.7631578947368421]
accuracy
accuracy

Working on... AMGN
/home/andrea/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
610.5402296585327
Window: 0
[[275 319]
 [274 286]]
F1,score 0.49098712446351933
Accuracy: 0.4861351819757366
372.7593720314938
Window: 4
[[237 361]
 [213 342]]
F1,score 0.5437201907790143
Accuracy: 0.5021682567215958
1000.0
Window: 5
[[165 434]
 [176 378]]
F1,score 0.5534407027818448
Accuracy: 0.47094535993061576
10000.0
Window: 25
[[113 430]
 [ 92 514]]
F1,score 0.6632258064516129
Accuracy: 0.5456919060052219
2.782559402207126
Window: 40
[[ 66 461]
 [  1 618]]
F1,score 0.7279151943462898
Accuracy: 0.5968586387434555
2.6826957952797255
Window: 125
[[141 376]
 [  0 612]]
F1,score 0.765
Accuracy: 0.66696191319752
84.83428982440716
Window: 170
[[175 389]
 [  2 554]]
F1,score 0.7391594396264176
Accuracy: 0.6508928571428572
0.3593813663804626
Window: 250
[[201 397]
 [ 18 488]]
F1,score 0.7016534867002155
Accuracy: 0.6240942028985508
0.000774263682681127
Window: 290
[[101 552]
 [ 15 428]]
F1,score 0.6015460295151089
Accuracy: 0.48266423357664234
1.0
Window: 350
[[323 392]
 [  0 369]]
F1,score 0.6530973451327434
Accuracy: 0.6383763837638377
1.0
Window: 400
[[388 309]
 [  0 377]]
F1,score 0.7093132643461901
Accuracy: 0.7122905027932961
1.0
Window: 450
[[404 298]
 [  0 362]]
F1,score 0.7084148727984345
Accuracy: 0.7199248120300752
[0, 0, 0, 0, 0, 0, 0, 25, 25, 25, 25, 25]
[0.4861351819757366, 0.5021682567215958, 0.47094535993061576, 0.5456919060052219, 0.5968586387434555, 0.66696191319752, 0.6508928571428572, 0.6240942028985508, 0.48266423357664234, 0.6383763837638377, 0.7122905027932961, 0.7199248120300752]
accuracy
accuracy

Working on... TXN
/home/andrea/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
2.6826957952797255
Window: 0
[[436 132]
 [443 143]]
F1,score 0.3321718931475029
Accuracy: 0.5017331022530329
1.637893706954064
Window: 4
[[320 228]
 [342 263]]
F1,score 0.47992700729927
Accuracy: 0.5056374674761491
7.196856730011519
Window: 5
[[369 166]
 [378 240]]
F1,score 0.46875000000000006
Accuracy: 0.5281873373807459
10000.0
Window: 25
[[274 233]
 [349 293]]
F1,score 0.5017123287671232
Accuracy: 0.4934725848563969
2.782559402207126
Window: 40
[[337 153]
 [501 155]]
F1,score 0.32157676348547715
Accuracy: 0.4293193717277487
610.5402296585327
Window: 125
[[189 268]
 [157 515]]
F1,score 0.7079037800687286
Accuracy: 0.6235606731620903
1291.5496650148827
Window: 170
[[190 322]
 [ 82 526]]
F1,score 0.7225274725274725
Accuracy: 0.6392857142857142
227.58459260747887
Window: 250
[[240 331]
 [ 13 520]]
F1,score 0.7514450867052023
Accuracy: 0.6884057971014492
227.58459260747887
Window: 290
[[292 325]
 [ 57 422]]
F1,score 0.6884176182707994
Accuracy: 0.6514598540145985
372.7593720314938
Window: 350
[[329 300]
 [ 13 442]]
F1,score 0.7385129490392648
Accuracy: 0.7112546125461254
166.81005372000558
Window: 400
[[360 248]
 [ 63 403]]
F1,score 0.7215756490599821
Accuracy: 0.7104283054003724
138.94954943731375
Window: 450
[[393 157]
 [ 14 500]]
F1,score 0.8539709649871904
Accuracy: 0.8392857142857143
[0, 0, 0, 0, 0, 0, 0, 25, 25, 25, 50, 50]
[0.5017331022530329, 0.5056374674761491, 0.5281873373807459, 0.4934725848563969, 0.4293193717277487, 0.6235606731620903, 0.6392857142857142, 0.6884057971014492, 0.6514598540145985, 0.7112546125461254, 0.7104283054003724, 0.8392857142857143]
accuracy
accuracy

Working on... AVGO
/home/andrea/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
0.0001
Window: 0
[[325 227]
 [356 246]]
F1,score 0.4576744186046512
Accuracy: 0.4948006932409012
11.787686347935873
Window: 4
[[262 321]
 [245 325]]
F1,score 0.5345394736842105
Accuracy: 0.5091066782307025
10000.0
Window: 5
[[225 354]
 [238 336]]
F1,score 0.5316455696202531
Accuracy: 0.4865568083261058
2.782559402207126
Window: 25
[[101 511]
 [ 84 453]]
F1,score 0.603597601598934
Accuracy: 0.4821583986074848
7.196856730011519
Window: 40
[[ 21 538]
 [  4 583]]
F1,score 0.6826697892271664
Accuracy: 0.5270506108202443
1.637893706954064
Window: 125
[[ 44 575]
 [  0 510]]
F1,score 0.6394984326018809
Accuracy: 0.49069973427812225
227.58459260747887
Window: 170
[[131 477]
 [  5 507]]
F1,score 0.6778074866310161
Accuracy: 0.5696428571428571
1.0
Window: 250
[[162 463]
 [ 19 460]]
F1,score 0.6562054208273894
Accuracy: 0.5634057971014492
1.0
Window: 290
[[157 498]
 [  0 441]]
F1,score 0.6391304347826087
Accuracy: 0.5456204379562044
1.0
Window: 350
[[124 544]
 [  0 416]]
F1,score 0.6046511627906976
Accuracy: 0.4981549815498155
1.0
Window: 400
[[112 542]
 [  0 420]]
F1,score 0.6078147612156296
Accuracy: 0.49534450651769085
1.637893706954064
Window: 450
[[116 623]
 [  0 325]]
F1,score 0.5106048703849175
Accuracy: 0.4144736842105263
[0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 25, 25]
[0.4948006932409012, 0.5091066782307025, 0.4865568083261058, 0.4821583986074848, 0.5270506108202443, 0.49069973427812225, 0.5696428571428571, 0.5634057971014492, 0.5456204379562044, 0.4981549815498155, 0.49534450651769085, 0.4144736842105263]
accuracy
accuracy

Working on... PYPL
/home/andrea/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
10000.0
Window: 0
[[271 294]
 [307 282]]
F1,score 0.48412017167381977
Accuracy: 0.47920277296360486
0.0001
Window: 4
[[345 237]
 [310 262]]
F1,score 0.4892623716153128
Accuracy: 0.5259965337954939
0.0001
Window: 5
[[359 208]
 [347 239]]
F1,score 0.462729912875121
Accuracy: 0.5186470078057241
0.005994842503189409
Window: 25
[[300 219]
 [381 249]]
F1,score 0.453551912568306
Accuracy: 0.47780678851174935
11.787686347935873
Window: 40
[[264 261]
 [319 302]]
F1,score 0.5101351351351351
Accuracy: 0.493891797556719
1.637893706954064
Window: 125
[[455  82]
 [525  67]]
F1,score 0.1808367071524966
Accuracy: 0.46235606731620904
10000.0
Window: 170
[[305 242]
 [137 436]]
F1,score 0.6970423661071143
Accuracy: 0.6616071428571428
10000.0
Window: 250
[[361 238]
 [ 63 442]]
F1,score 0.7459915611814346
Accuracy: 0.7273550724637681
10000.0
Window: 290
[[331 303]
 [ 37 425]]
F1,score 0.7142857142857143
Accuracy: 0.6897810218978102
10000.0
Window: 350
[[493 192]
 [ 50 349]]
F1,score 0.7425531914893617
Accuracy: 0.7767527675276753
10000.0
Window: 400
[[448 234]
 [ 88 304]]
F1,score 0.6537634408602151
Accuracy: 0.7001862197392924
10000.0
Window: 450
[[559 187]
 [139 179]]
F1,score 0.5233918128654971
Accuracy: 0.693609022556391
[0, 0, 0, 0, 0, 0, 0, 25, 25, 25, 25, 25]
[0.47920277296360486, 0.5259965337954939, 0.5186470078057241, 0.47780678851174935, 0.493891797556719, 0.46235606731620904, 0.6616071428571428, 0.7273550724637681, 0.6897810218978102, 0.7767527675276753, 0.7001862197392924, 0.693609022556391]
accuracy
accuracy

Working on... GILD
10000.0
Window: 0
[[183 403]
 [162 406]]
F1,score 0.5896877269426288
Accuracy: 0.5103986135181976
21.54434690031882
Window: 4
[[ 72 512]
 [ 47 522]]
F1,score 0.6512788521522146
Accuracy: 0.5151777970511708
11.787686347935873
Window: 5
[[ 38 557]
 [ 21 537]]
F1,score 0.6501210653753027
Accuracy: 0.4986990459670425
2.6826957952797255
Window: 25
[[ 12 606]
 [  1 530]]
F1,score 0.6358728254349131
Accuracy: 0.47171453437771976
84.83428982440716
Window: 40
[[ 57 543]
 [ 23 523]]
F1,score 0.6488833746898263
Accuracy: 0.506108202443281
0.3593813663804626
Window: 125
[[ 71 615]
 [  0 443]]
F1,score 0.5902731512325117
Accuracy: 0.4552701505757307
10000.0
Window: 170
[[271 466]
 [ 28 355]]
F1,score 0.5897009966777409
Accuracy: 0.5589285714285714
10000.0
Window: 250
[[263 550]
 [ 11 280]]
F1,score 0.4995539696699376
Accuracy: 0.49184782608695654
0.0001
Window: 290
[[633 215]
 [116 132]]
F1,score 0.4436974789915966
Accuracy: 0.697992700729927
1291.5496650148827
Window: 350
[[474 419]
 [117  74]]
F1,score 0.21637426900584797
Accuracy: 0.5055350553505535
0.000774263682681127
Window: 400
[[934  10]
 [129   1]]
F1,score 0.014184397163120569
Accuracy: 0.8705772811918063
1.0
Window: 450
[[354 623]
 [  0  87]]
F1,score 0.21831869510664992
Accuracy: 0.4144736842105263
[0, 0, 0, 0, 0, 0, 25, 50, 25, 25, 25, 25]
[0.5103986135181976, 0.5151777970511708, 0.4986990459670425, 0.47171453437771976, 0.506108202443281, 0.4552701505757307, 0.5589285714285714, 0.49184782608695654, 0.697992700729927, 0.5055350553505535, 0.8705772811918063, 0.4144736842105263]
accuracy
accuracy

Working on... COST
4.39397056076079
Window: 0
[[560   5]
 [575  14]]
F1,score 0.04605263157894736
Accuracy: 0.49740034662045063
10000.0
Window: 4
[[305 244]
 [321 284]]
F1,score 0.5013239187996469
Accuracy: 0.5103986135181976
227.58459260747887
Window: 5
[[415 141]
 [441 156]]
F1,score 0.3489932885906041
Accuracy: 0.49522983521248914
166.81005372000558
Window: 25
[[386 127]
 [467 169]]
F1,score 0.3626609442060086
Accuracy: 0.4830287206266319
1291.5496650148827
Window: 40
[[389 125]
 [442 190]]
F1,score 0.4012671594508976
Accuracy: 0.5052356020942408
2.6826957952797255
Window: 125
[[419   2]
 [694  14]]
F1,score 0.03867403314917127
Accuracy: 0.38352524357838796
0.0001
Window: 170
[[333  84]
 [565 138]]
F1,score 0.2983783783783784
Accuracy: 0.4205357142857143
10000.0
Window: 250
[[327  51]
 [541 185]]
F1,score 0.3846153846153847
Accuracy: 0.463768115942029
10000.0
Window: 290
[[289 111]
 [575 121]]
F1,score 0.2607758620689655
Accuracy: 0.3740875912408759
0.046415888336127774
Window: 350
[[320   3]
 [752   9]]
F1,score 0.02328589909443726
Accuracy: 0.30350553505535055
10000.0
Window: 400
[[220  74]
 [631 149]]
F1,score 0.2971086739780658
Accuracy: 0.3435754189944134
84.83428982440716
Window: 450
[[293  43]
 [643  85]]
F1,score 0.1985981308411215
Accuracy: 0.35526315789473684
[0, 0, 0, 0, 0, 0, 0, 50, 50, 50, 50, 50]
[0.49740034662045063, 0.5103986135181976, 0.49522983521248914, 0.4830287206266319, 0.5052356020942408, 0.38352524357838796, 0.4205357142857143, 0.463768115942029, 0.3740875912408759, 0.30350553505535055, 0.3435754189944134, 0.35526315789473684]
accuracy
accuracy

Working on... QCOM
/home/andrea/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
1.637893706954064
Window: 0
[[312 265]
 [288 290]]
F1,score 0.5119152691968226
Accuracy: 0.5212121212121212
2.782559402207126
Window: 4
[[300 282]
 [231 341]]
F1,score 0.5707112970711297
Accuracy: 0.5554592720970537
11.787686347935873
Window: 5
[[300 268]
 [228 358]]
F1,score 0.5907590759075908
Accuracy: 0.5701906412478336
31.622776601683793
Window: 25
[[124 443]
 [ 46 537]]
F1,score 0.6871401151631478
Accuracy: 0.5747826086956521
31.622776601683793
Window: 40
[[ 73 492]
 [ 32 550]]
F1,score 0.6773399014778325
Accuracy: 0.5431560592850916
11.787686347935873
Window: 125
[[ 40 475]
 [ 12 603]]
F1,score 0.7123449497932663
Accuracy: 0.5690265486725664
1.0
Window: 170
[[ 38 534]
 [ 41 508]]
F1,score 0.6385920804525456
Accuracy: 0.4870651204281891
0.000774263682681127
Window: 250
[[281 378]
 [154 292]]
F1,score 0.5232974910394265
Accuracy: 0.5185520361990951
138.94954943731375
Window: 290
[[147 542]
 [  1 407]]
F1,score 0.599852616064849
Accuracy: 0.5050136736554239
0.046415888336127774
Window: 350
[[247 508]
 [ 14 316]]
F1,score 0.5476603119584055
Accuracy: 0.5188940092165899
0.046415888336127774
Window: 400
[[158 641]
 [  0 276]]
F1,score 0.4626990779547359
Accuracy: 0.4037209302325581
0.046415888336127774
Window: 450
[[192 660]
 [  0 213]]
F1,score 0.39226519337016574
Accuracy: 0.38028169014084506
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 25]
[0.5212121212121212, 0.5554592720970537, 0.5701906412478336, 0.5747826086956521, 0.5431560592850916, 0.5690265486725664, 0.4870651204281891, 0.5185520361990951, 0.5050136736554239, 0.5188940092165899, 0.4037209302325581, 0.38028169014084506]
accuracy
accuracy

